# Aim
The aim of this analysis is to determine credit risk based on customer behaviour regarding engagement with financial products as well as a variety of demographic attributes that may be informative.

It's worth noting that the dataset already contains information about credit risk

## Data source

In this analysis I'm using a **Kaggle credit risk dataset** described [here](https://www.kaggle.com/datasets/praveengovi/credit-risk-classification-dataset)


The data is structured in two csv files:

:::{}
1. File 1 - **Payment data** contains customer's credit card payment history and has the following fields:
- id: customer id
- OVD_t1: number of times overdue type 1
- OVD_t2: number of times overdue type 2
- OVD_t3: number of times overdue type 3
- OVD_sum: total overdue days
- pay_normal: number of times normal payment
- prod_code: credit product code
- prod_limit: credit limit of product
- update_date: account update date
- new_balance: current balance of product
- highest_balance: highest balance in history
- report_date: date of recent payment
:::

:::{}
2. File 2 - **Customer data** which contains demographic data and category attributes that have been anonymised as follows:
- id
- label:  if it equals 1 it indicates the customer is high risk, if it equals 0 the customer is low risk
- fea_1
- fea_2
- fea_3
- fea_4
- fea_5
- fea_6
- fea_7
- fea_8
- fea_9
- fea_10
- fea_11
:::

## Methodology


```{mermaid}
flowchart LR
  A[Input data] --> B(EDA)
  B --> C{Analysis proper}
  C --> D[ML methods]
  D --> F{Logistic Regression}
  D --> G{Naive Bayes}
  D --> H{Decision Tree Classifier}
  D --> I{XGBoost}
  C --> E[Neural Network]
  E --> J{Accuracy assessment}
  F --> J{Accuracy assessment}
  G --> J{Accuracy assessment}
  H --> J{Accuracy assessment}
  I --> J{Accuracy assessment}
  J --> K{Reporting}
  K --> L{Select best two methods}
  L --> M{Compare common and unique predictions}
```


### Input data pre-processing

In [ ]:
#| echo: false

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
import plotnine as p9
from plotnine import ggplot, aes, geom_tile, geom_violin, geom_boxplot, geom_label, scale_fill_cmap, scale_fill_gradient, scale_fill_manual, theme_minimal, labs, element_text, facet_wrap
import plydata.cat_tools as cat
import os

# Load the data
df1 = pd.read_csv ('~/Desktop/DS/kaggle_projects/Credit_Risk_Classification/data/customer_data.csv')
df2 = pd.read_csv ('~/Desktop/DS/kaggle_projects/Credit_Risk_Classification/data/payment_data.csv')

# Create a temporary DataFrame for renaming columns
df1_renamed = df1.copy()

# Rename columns in df1_renamed
df1_renamed.columns = df1_renamed.columns.str.replace(r'fea_(\d+)', r'feature_\1', regex=True)

# Select columns to keep from the original df1
columns_to_keep = ['id', 'label']

# Select only the renamed feature columns from df1_renamed
renamed_feature_columns = df1_renamed.filter(like='feature_')

# Concatenate the 'id', 'label', and renamed feature columns
df1_final = pd.concat([df1[columns_to_keep], renamed_feature_columns], axis=1)
df1_final

df1_final.isnull().sum()
# we are expencting one with NAs (feature_2)
df2.isnull().sum()
# For df2 we are expecting NAs on prod_limit and highest balance

# Merge df1_final and df2 on id
merged_df = df1_final.merge(df2, on='id')

# Drop the two columns for which we have no reasonable way of finding out cause for NA
merged_df.drop(columns=['update_date','report_date'], inplace=True)

# Check NAs (there should be only three features with NAs)
merged_df.isnull().sum()

# Let's impute those missing values for feature_2, prod_limit, and highest balance
from sklearn.impute import KNNImputer

impute = KNNImputer(n_neighbors= 5)
cols = ['prod_limit','highest_balance','feature_2']
merged_df[cols] = merged_df[cols].round()
for i in cols:
    merged_df[i] = impute.fit_transform(merged_df[[i]])

x = merged_df.drop(columns=['label'],axis=1)
y = merged_df['label']
merged_df.drop(columns=['label'], axis=1, inplace=True)

### Note:
After some standard pre-processing (renaming columns, concatenating data frames, and checking for missing data) I decided that data imputation was not appropriate while low credit risk customers might pay their full balance or higher amounts consistently on the same date, high risk customers would exhibit far more variance on both the date of balance closing and last payments made. I personally feel the variable 'pay_normal' (i.e. the number of times a customer does not default on payments) captures this information in a discretised manner that does not rely on following a Poisson distribution of events.

In short, I have decided that dropping update_date, and report_date variables was justified. I did however use KNN imputation to fill missing values for feature_2, prod_limit (the credit limit), and highest_balance features, the data for feature 2 appears to be normally distributed, while the prod_limit appears to be close to normally distributed but has a small number of outliers. The highest_balance variable exhibits most values centered close to zero but extremely long tail all the way out  to 150,000,000. 

##### No missing values present after pre-processing

In [ ]:
merged_df.info()

It would be good to visualise how the different variables correlate with one another.

*Technical note: The plot was generated with plotnine following Tidyverse principles which I find to be superior to Python libraries*


In [ ]:
#| echo: false
#| fig-cap: "Correlation between different variables in the credit risk customer dataset"

import plotnine as p9
from plotnine import aes, geom_tile, geom_label, scale_fill_cmap, theme_minimal, labs, element_text
import plydata.cat_tools as cat

tidy_corr = merged_df \
    .corr() \
    .melt(
        ignore_index=False,
    ) \
    .reset_index() \
    .set_axis(
        labels = ["var1", "var2", "value"],
        axis = 1
    ) \
    .assign(lab_text = lambda x: np.round(x['value'], 2)) \
    .assign(
        var1 = lambda x: cat.cat_inorder(x['var1']),
        var2 = lambda x:
             cat.cat_rev(
                 cat.cat_inorder(x['var2'])
             )
    )

tidy_corr

# Filter tidy_corr to exclude values that are 1.0 or below 0.1
tidy_corr_filtered = tidy_corr[(tidy_corr['value'] > 0.1) & (tidy_corr['value'] < 1.0)]

(p9.ggplot(
    mapping=p9.aes("var1", "var2", fill="value"),
    data=tidy_corr
) +
p9.geom_tile(alpha=0.8) +  # Adjust alpha
p9.geom_label(
    p9.aes(label="lab_text"),
    fill="white",
    size=8,
    data=tidy_corr_filtered  # Apply filter to geom_label
) +
scale_fill_cmap(name="Correlation", cmap="viridis") +  # Use viridis colormap
theme_minimal() +
labs(
    title="Credit Risk | Correlation Matrix Merged Data",
    x="", y=""
) +
p9.theme(
    axis_text_x=element_text(rotation=70, hjust=1),
    figure_size=(8, 6)
))

## Analyses

First thing here is to store the label used to classify a customer as high or low credit risk and store it
as the response variable for all our models while keeping everything else as our predictor variables


### ML methods

Four methods will be used, Logistic Regression (LR), Naive Bayes (NB), a Decision Tree Classifier (DT), and XGBoost (XGB).
The dataset is split into training and test sets, **70%** is kept for the training set and **30%** will be used as a test set.

Hyperparameter tuning was performed for both the DT and XGB models

The code is not shown but the prediction accuracy is reported below


In [ ]:
#| echo: false
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

# going for a 30% split
X_train,X_test,Y_train,Y_test = train_test_split(x,y, test_size= 0.3)

################################
##### Logistic Regression ######
lr = LogisticRegression()
lr.fit(X_train,Y_train)
Y_pred_lr = lr.predict(X_test)



# Calculate the accuracy score between prediction and the test data
score_lr = round(accuracy_score(Y_pred_lr,Y_test)*100, 2)

###############################
###### Naive Bayes model ######
nb = GaussianNB()
nb.fit(X_train,Y_train)
Y_pred_nb = nb.predict(X_test)


# Naive Bayes accuracy score
score_nb = round(accuracy_score(Y_pred_nb,Y_test)*100, 2)

######################################
###### Decision Tree Classifier ######

# initialise the max accuracy object for hyperparameter tuning
max_accuracy = 0

# hyperparameter tuning loop
for x in range(200):
    dt = DecisionTreeClassifier(random_state=x)
    dt.fit(X_train,Y_train)
    Y_pred_dt = dt.predict(X_test)
    current_accuracy = round(accuracy_score(Y_pred_dt,Y_test)*100, 2)
    if(current_accuracy>max_accuracy):
        max_accuracy = current_accuracy
        best_x = x

print("Best random state after hyperparameter tuning for the Decision Tree Classifier") 
print(best_x)

# train the model with the best random state
dt = DecisionTreeClassifier(random_state=best_x)
dt.fit(X_train,Y_train)
Y_pred_dt = dt.predict(X_test)

# calculate the accuracy
score_dt = round(accuracy_score(Y_pred_dt,Y_test)*100, 2)

#####################
###### XGBoost XXXXXX

# Save the initial XGBoost model
import pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(dt, f)
import xgboost as xgb

# Initialize the max accuracy object for hyperparameter tuning
max_accuracy_xgb = 0

# Hyperparameter tuning loop for XGBoost
for x in range(200):
    xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=x)
    xgb_model.fit(X_train, Y_train)
    Y_pred_xgb = xgb_model.predict(X_test)
    current_accuracy_xgb = round(accuracy_score(Y_pred_xgb, Y_test) * 100, 2)
    if current_accuracy_xgb > max_accuracy_xgb:
        max_accuracy_xgb = current_accuracy_xgb
        best_x_xgb = x

print("Best random state after hyperparameter tuning for XGBoost")       
print(best_x_xgb)


# train the model with the best random state
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=0)
xgb_model.fit(X_train, Y_train)
Y_pred_xgb = xgb_model.predict(X_test)


# calculate the accuracy
score_xgb = round(accuracy_score(Y_pred_xgb,Y_test)*100, 2)

# save the trained XGBoost model
import pickle
with open('model2.pkl', 'wb') as f:
    pickle.dump(xgb_model, f)

### Accuracy of the ML model predictions

In [ ]:
#| echo: false
scores = [score_lr,score_nb,score_dt,score_xgb]
algorithms = ["Logistic Regression","Naive Bayes","Decision Tree","XGBoost"]    

for i in range(len(algorithms)):
    print("The accuracy score achieved using "+algorithms[i]+" is: "+str(scores[i])+" %")

### Neural Network approach

I have used a NN model with a full connected hidden layer containing 11 units and ReLu as activation function. The output layer is configured for binary classification.

The model was trained for 3,000 epochs before carrying out predictions on the test set. 


In [ ]:
#| echo: false
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense


# Build the NN model
model = Sequential()
# 11 units on a fully connected hidden layer, the number of input dimensions is the features in our merged_df
model.add(Dense(11, activation='relu', input_dim=21))
# output layer for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the NN model
# configure loss method for binary classification
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, Y_train, epochs=3000)

# Make predictions on the test data with our trained model
Y_pred_nn = model.predict(X_test)

#### Accuracy of the NN model predictions


In [ ]:
#| echo: false
rounded = [round(x[0]) for x in Y_pred_nn]
Y_pred_nn = rounded
score_nn = round(accuracy_score(Y_pred_nn, Y_test) * 100, 2)
print("The accuracy score achieved using Neural Network is: " + str(score_nn) + " %")

#### Reporting the results of our models

In [ ]:
#| echo: false
from plotnine import ggplot, aes, geom_bar, theme_minimal, labs, scale_fill_cmap
import matplotlib.cm as cm
import numpy as np

scores = [score_lr,score_nb,score_dt,score_xgb,score_nn]
algorithms = ["Logistic Regression","Naive Bayes","Decision Tree","XGBoost","Neural Network"]    

# Define the custom order (I find it is more tasteful to organise the categories according to their
# value, it seems to be the default behaviour of bar plots to order categories alphabetically/alphanumerically
# regardless of the arguments passed (although in R the forcats library gets around this superbly))
custom_order = ["XGBoost", "Decision Tree", "Logistic Regression", "Neural Network", "Naive Bayes"]

# Create a DataFrame for plotting
df = pd.DataFrame({"Algorithm": algorithms, "Score": scores})

# Reorder the DataFrame based on the custom order
df["Algorithm"] = pd.Categorical(df["Algorithm"], categories=custom_order, ordered=True)
df = df.sort_values("Algorithm")
# Sort the DataFrame by score in descending order
#df = df.sort_values(by="Score", ascending=False)

# Assign specified colors to algorithms
# I couldn't get this to simply sample 5 colours from viridis and then assign them
# to each algorithm rather than assign the colours to the accuracy scores so
# if you need to generate more in order to fit say 6 models, here's a resource
# for that.
# https://waldyrious.net/viridis-palette-generator/

colour_dict = {
    "XGBoost": "#440154",
    "Decision Tree": "#3b528b",
    "Logistic Regression": "#21918c",
    "Neural Network": "#5ec962",
    "Naive Bayes": "#fde725"
}

# Add a new 'Colour' column based on algorithm order
df["Colour"] = df["Algorithm"].map(colour_dict)

p = (ggplot(df, aes(x="Algorithm", y="Score", fill="Colour"))
    + geom_bar(stat="identity")
    + theme_minimal()
    + labs(title="Algorithm Scores", x="Algorithm", y="Accuracy")
    + scale_fill_manual(values=df["Colour"].tolist(), guide=False)
)

print(p)

#### Selecting the best methods based on the accuracy scores

The reason why one would want to consider selecting the results of more than one method are many. For one, the way a given classifier method might choose a given outcome may be affected by specific statistical quirks of each method, for closely scoring methods one may want to see what predicted IDs are common to both methods and which ones aren't in order to better understand what parameters might be more relevant to each model.


In [ ]:
#| echo: false
# Predict labels using XGBoost
xgb_predicted_labels = xgb_model.predict(X_test)

# Predict labels using Decision Tree
dt_predicted_labels = dt.predict(X_test)

# Filter rows where XGBoost predicted high credit risk (1)
high_risk_cases_xgb = X_test[xgb_predicted_labels == 1]

# Filter rows where Decision Tree predicted high credit risk (1)
high_risk_cases_dt = X_test[dt_predicted_labels == 1]

# Now you have two DataFrames: high_risk_cases_xgb and high_risk_cases_dt
# These DataFrames contain cases where the respective models predicted high credit risk.

# Concatenate both DataFrames vertically
common_high_risk_cases = pd.concat([high_risk_cases_xgb, high_risk_cases_dt], ignore_index=True)

# Drop duplicates based on the 'id' column
common_high_risk_cases = common_high_risk_cases.drop_duplicates(subset='id')

# Print the resulting DataFrame
#print(common_high_risk_cases)


# Create DataFrames to store the IDs that were predicted by one algorithm but not the other
xgb_missing_ids = X_test.copy()
xgb_missing_ids['model'] = 'xgb'
xgb_missing_ids = xgb_missing_ids[~xgb_predicted_labels.astype(bool)]

dt_missing_ids = X_test.copy()
dt_missing_ids['model'] = 'dt'
dt_missing_ids = dt_missing_ids[~dt_predicted_labels.astype(bool)]

# Concatenate both DataFrames vertically
missing_ids = pd.concat([xgb_missing_ids, dt_missing_ids], ignore_index=True)

# Drop duplicates based on the 'id' column
missing_ids = missing_ids.drop_duplicates(subset='id')

#### Visualising differences between the predicted customers common to both models and those that are unique to either model

##### Plot all high risk cases predicted by both XGBoost and DT models


In [ ]:
#| echo: false
import plotnine as p9
from plotnine import aes, geom_tile, geom_label, scale_fill_cmap, theme_minimal, labs, element_text
import plydata.cat_tools as cat

tidy_corr = df_com \
    .corr() \
    .melt(
        ignore_index=False,
    ) \
    .reset_index() \
    .set_axis(
        labels = ["var1", "var2", "value"],
        axis = 1
    ) \
    .assign(lab_text = lambda x: np.round(x['value'], 2)) \
    .assign(
        var1 = lambda x: cat.cat_inorder(x['var1']),
        var2 = lambda x:
             cat.cat_rev(
                 cat.cat_inorder(x['var2'])
             )
    )

tidy_corr

# Filter tidy_corr to exclude values that are 1.0 or below 0.1
tidy_corr_filtered = tidy_corr[(tidy_corr['value'] > 0.1) & (tidy_corr['value'] < 1.0)]

(p9.ggplot(
    mapping=p9.aes("var1", "var2", fill="value"),
    data=tidy_corr
) +
p9.geom_tile(alpha=0.8) +  # Adjust alpha
p9.geom_label(
    p9.aes(label="lab_text"),
    fill="white",
    size=8,
    data=tidy_corr_filtered  # Apply filter to geom_label
) +
scale_fill_cmap(name="Correlation", cmap="viridis") +  # Use viridis colormap
theme_minimal() +
labs(
    title="Correlation Matrix ML predicted high risk cases",
    x="", y=""
) +
p9.theme(
    axis_text_x=element_text(rotation=70, hjust=1),
    figure_size=(8, 6)
)) 

##### And those predicted by XGBoost but not the Decision Tree Classifier model


In [ ]:
#| echo: false
# Now the missing cases for the DT model (i.e. the ones predicted by XGBoost but not DT)

tidy_corr = df_mis_dt \
    .corr() \
    .melt(
        ignore_index=False,
    ) \
    .reset_index() \
    .set_axis(
        labels = ["var1", "var2", "value"],
        axis = 1
    ) \
    .assign(lab_text = lambda x: np.round(x['value'], 2)) \
    .assign(
        var1 = lambda x: cat.cat_inorder(x['var1']),
        var2 = lambda x:
             cat.cat_rev(
                 cat.cat_inorder(x['var2'])
             )
    )

tidy_corr

# Filter tidy_corr to exclude values that are 1.0 or below 0.1
tidy_corr_filtered = tidy_corr[(tidy_corr['value'] > 0.1) & (tidy_corr['value'] < 1.0)]

(p9.ggplot(
    mapping=p9.aes("var1", "var2", fill="value"),
    data=tidy_corr
) +
p9.geom_tile(alpha=0.8) +  # Adjust alpha
p9.geom_label(
    p9.aes(label="lab_text"),
    fill="white",
    size=8,
    data=tidy_corr_filtered  # Apply filter to geom_label
) +
scale_fill_cmap(name="Correlation", cmap="viridis") +  # Use viridis colormap
theme_minimal() +
labs(
    title="Correlation Matrix XGBoost unique high risk cases",
    x="", y=""
) +
p9.theme(
    axis_text_x=element_text(rotation=70, hjust=1),
    figure_size=(8, 6)
)) 

##### Finally those predictions made by the Decision Tree Classifier model but not XGBoost

In [ ]:
#| echo: false
# And for those predicted by DT but not XGBoost
tidy_corr = df_mis_xgb \
    .corr() \
    .melt(
        ignore_index=False,
    ) \
    .reset_index() \
    .set_axis(
        labels = ["var1", "var2", "value"],
        axis = 1
    ) \
    .assign(lab_text = lambda x: np.round(x['value'], 2)) \
    .assign(
        var1 = lambda x: cat.cat_inorder(x['var1']),
        var2 = lambda x:
             cat.cat_rev(
                 cat.cat_inorder(x['var2'])
             )
    )

tidy_corr

# Filter tidy_corr to exclude values that are 1.0 or below 0.1
tidy_corr_filtered = tidy_corr[(tidy_corr['value'] > 0.1) & (tidy_corr['value'] < 1.0)]

(p9.ggplot(
    mapping=p9.aes("var1", "var2", fill="value"),
    data=tidy_corr
) +
p9.geom_tile(alpha=0.8) +  # Adjust alpha
p9.geom_label(
    p9.aes(label="lab_text"),
    fill="white",
    size=8,
    data=tidy_corr_filtered  # Apply filter to geom_label
) +
scale_fill_cmap(name="Correlation", cmap="viridis") +  # Use viridis colormap
theme_minimal() +
labs(
    title="Correlation Matrix DT unique high risk cases",
    x="", y=""
) +
p9.theme(
    axis_text_x=element_text(rotation=70, hjust=1),
    figure_size=(8, 6)
)) 

We can see that the features that matter to predict high risk are the same for both XGBoost and DT models. However, there are some differences in the correlation values possibly arising from differences in sensitivity between ensemble models (XGBoost) and other classifier models such as decision tree classifiers. Whether the sensitivity threshold in making these predictions should be tweaked or not would depend on how much risk can be tolerated but both models perform well, although XGBoost shows marginally better performance.